# Set up

## Mounting drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# make sure the a bot that's not a bot is in your drive's root folder
%cd "/content/drive/MyDrive/bot-that's-not-a-bot/models"

/content/drive/.shortcut-targets-by-id/1JM2zeyfEBunPH1XKNRbCKtL__BLZXR8Y/bot-that's-not-a-bot/models


## Imports

In [3]:
!pip install bert-for-tf2 >> /dev/null
!pip install transformers --quiet
!pip install tensorflow==2.3.4 >> /dev/null
!pip install pandas==1.3.2 >> /dev/null

In [4]:
from tensorflow.keras.models import load_model
from bert import BertModelLayer
from bert.tokenization.bert_tokenization import FullTokenizer
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, pipeline
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

## Load models

In [5]:
bert = load_model('Bert.h5', custom_objects={'BertModelLayer': BertModelLayer})
bart_large = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')

## Constants

In [6]:
MAX_SEQ_LEN = bert.layers[0].input_shape[0][1]

class Response():
  def __init__(self, custom, value=""):
    '''
    Args:
      custom: if true then the subintent requires a custom response (have to be modified in the GUI function)
      value: the response for the subintent
    '''
    self.custom = custom
    self.value = value

# the intents that can be classified by bert 
CLASSES = ['food', 'get help', 'make call', 'out_of_scope', 'send text', 'visitors']

# subintents that can be classified by bart-large (zero-shot learning) 
SUBCLASSES = {"food":{"when is meal": Response(False, "Your next meal is in 2 hours"),
                      "get food menu": Response(False, "Here is the food menu"),
                      "get drink menu": Response(False, "Here is the drink menu"), 
                      "order food or drink": Response(True)},
              "get help":{"get medical assistance": Response(False, "Calling for medical assistance..."),
                          "get 911": Response(False, "Calling 911..."),
                          "help changing clothes": Response(False, "Getting the nurse to help you get dressed..."),
                          "help going to bathroom": Response(False, "Getting the nurse to help you to the bathroom...")},
              "make call":{"make call": Response(True)},
              "send text": {"send text": Response(True)},
              "visitors":{"visiting hours": Response(False, "Visiting hours are set for 3 to 5"),
                          "who is visiting": Response(False, "The scheduled visitors are Josue, Michael, Ivy, and David"),
                          "visiting schedule":Response(False, "Here is the visiting schedule")}}

 

# the food that are offered in provided at the hospital in a particular day
FOODS = ["spaghetti", "burritos", "nacho cheese", "bread", "fruit", "soup",
        "salad", "meatloaf", "pizza", "milk", "juice", "tea", "coffee", "lemonade", 
        "jelly", "chips", "hamburger", "yogurt", "eggs", "water", "pasta", "bagel", "tacos"]

# contacts of the patient 
CONTACTS = ["mom", "dad", "brother", "sister", "william", "olivia", "emma", "ava", "charlotte", "sophia",
         "amelia", "isabella", "mia", "evelyn", "harper", "camila", "gianna",
         "abigail", "luna", "ella", "elizabeth", "sofia", "emily", "avery",
         "mila", "scarlett", "eleanor", "madison", "layla", "penelope", "aria",
         "chloe", "grace", "ellie", "nora", "hazel", "zoey", "riley", "victoria",
         "lily", "aurora", "violet", "nova", "hannah", "emilia", "zoe", "stella",
         "everly", "isla", "leah", "lillian", "addison", "willow", "lucy", "david"]

## Set up tokenizer for Bert and Bart-Large

In [7]:
# we have this file already in our data
bert_tokenizer = FullTokenizer(vocab_file="/content/drive/MyDrive/bot-that's-not-a-bot/data/vocab.txt")
bart_large_tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

## Helper functions

In [8]:
def get_bert_predictions(sentences, classes=CLASSES, model=bert, tokenizer=bert_tokenizer, max_seq_len=MAX_SEQ_LEN):
  '''
  Args:
    sentences (list): unprocessed texts to be passed in
    classes (list): the labels of the classes
  '''
  pred_tokens = map(tokenizer.tokenize, sentences)
  pred_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], pred_tokens)
  pred_token_ids = list(map(tokenizer.convert_tokens_to_ids, pred_tokens))
  pred_token_ids = map(lambda tids: tids +[0]*(max_seq_len-len(tids)),pred_token_ids)
  pred_token_ids = np.array(list(pred_token_ids))
  predictions = model.predict(pred_token_ids).argmax(axis=-1)
  return list(map(lambda index: classes[index], predictions))

In [9]:
def get_bart_large_predictions(sentences, classes, model=bart_large, tokenizer=bart_large_tokenizer):
  '''
  Args:
    sentences (list or str): unprocessed texts to be passed in
    classes (list): the labels of the classes
  '''
  zsc = pipeline(task='zero-shot-classification', tokenizer=tokenizer, model=model)
  return zsc(sequences=sentences, candidate_labels=classes, multi_label=False)

In [10]:
def find_name(names, sentence):
    '''
    Check if a name is in the list; if so return name otherwise "unknown"
    '''
    response = "unknown"
    for name in names:
        if name in sentence:
            response = name
            break
    return response

In [11]:
def GUI(sentence, classes=CLASSES, contacts=CONTACTS, foods=FOODS):
  '''
    Args: 
      sentence (str):
      contacts (list): contact names of the patient
      foods (list): food provided at the hospital in a particular day
  '''
  sentence = sentence.lower()
  intent = get_bert_predictions([sentence], classes=classes)[0]
  response = ""
  if intent == "out_of_scope":
    response = "Sorry, I did not understand"
    subintent = "out_of_scope"
  else: 
    subintent = get_bart_large_predictions(sentence, classes=[*SUBCLASSES[intent]])["labels"][0]
    response_obj = SUBCLASSES[intent][subintent]
    if response_obj.custom is True:
      if subintent == 'order food or drink':
        food = find_name(foods, sentence)
        if response=="unknown":
          response = "Sorry, that item is currently not on our menu. Here is our menu."
        else:
          response = f"Okay, I will order {food}" 
      elif subintent == 'make call':
        contact = find_name(contacts, sentence)
        if contact == "unknown":
          response = "Sorry, I couldn't find that person in your contacts."
        else:
          response = f"Calling {contact}..."
      elif subintent == 'send text':
        contact = find_name(contacts, sentence)
        if contact == "unknown":
          response = "Sorry, I couldn't find that person in your contacts."
        else:
          response = f"Drafing text to {contact}..."
    else:
      response = response_obj.value
  return dict(query=sentence, response=response, intent=intent, subintent=subintent)

# Gain intuitions on bart-large similarity scores

In [12]:
classes = ["visiting hours", "who is visiting", "visiting schedule"]
sentence = "When is Lucy visiting me"
get_bart_large_predictions(sentence, classes=classes)

{'labels': ['who is visiting', 'visiting schedule', 'visiting hours'],
 'scores': [0.43580302596092224, 0.34902748465538025, 0.2151694893836975],
 'sequence': 'When is Lucy visiting me'}

In [13]:
sentences = "I need help getting dressed"
classes = ["get medical assistance", "get 911", "help changing clothes", "help going to bathroom"]
get_bart_large_predictions(sentence, classes=classes)

{'labels': ['help changing clothes',
  'get medical assistance',
  'help going to bathroom',
  'get 911'],
 'scores': [0.373990923166275,
  0.2754444181919098,
  0.19381386041641235,
  0.15675075352191925],
 'sequence': 'When is Lucy visiting me'}

# GUI Demo

In [14]:
sentences = ["can I get tacos", 
             "can you open the window", 
             "call dad please",
             "send david a text", 
             "open the window would you",
             "can you get the nurse to help me change cloth",
             "get nurse nancy to help me go to the bathroom",
             "any one visiting me today",
             "when's today's visiting hours",
             "I'm hungry",
             "call the doctor please",
             "call 911 now",
             "can you get the nurse",
             "can you call nora",
             "text michael",
             "I want to drink lemonade"]

df = pd.DataFrame()
for sentence in sentences:
  output = GUI(sentence)
  df = df.append(output, ignore_index=True)
df = df.sort_values(by='intent', axis=0).reset_index(drop=True)
df = df.reindex(columns=['query', 'response', 'intent', 'subintent'])
s = df.style

In [17]:
slice_ = ['intent', 'subintent']
s.hide_index().set_properties(**{'opacity': '40%'}, subset=slice_)
cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000066; color: white;'
}
s.set_table_styles([cell_hover, index_names, headers])

query,response,intent,subintent
can i get tacos,"Okay, I will order tacos",food,order food or drink
i'm hungry,Your next meal is in 2 hours,food,when is meal
i want to drink lemonade,"Okay, I will order lemonade",food,order food or drink
get nurse nancy to help me go to the bathroom,Getting the nurse to help you to the bathroom...,get help,help going to bathroom
call the doctor please,Calling for medical assistance...,get help,get medical assistance
can you get the nurse,Calling for medical assistance...,get help,get medical assistance
call dad please,Calling dad...,make call,make call
call 911 now,"Sorry, I couldn't find that person in your contacts.",make call,make call
can you call nora,Calling nora...,make call,make call
can you open the window,"Sorry, I did not understand",out_of_scope,out_of_scope
